In [58]:
import pandas as pd

In [59]:
customers = pd.read_csv('./customers.csv')
types = pd.read_csv('./lda_distns.csv')
transaction = pd.read_csv('./database.csv')

In [60]:
names = [str(i) for i in range(25)]
types.columns = ['account_id']+names
types.head()

,account_id,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,576,0.118368,0.000212,0.007477,0.085284,0.005008,0.000212,0.012573,0.000212,0.168622,...,0.020308,0.042442,0.000212,0.081874,0.003604,0.116314,0.085687,0.181844,0.000212,0.003120
1,704,0.000859,0.027608,0.000860,0.000859,0.000859,0.000859,0.019092,0.000859,0.000860,...,0.000859,0.000859,0.000859,0.218398,0.000859,0.000859,0.188473,0.000859,0.000859,0.000860
2,192,0.000543,0.000543,0.392860,0.000543,0.000543,0.000543,0.000543,0.007720,0.000543,...,0.000543,0.000543,0.000543,0.249266,0.000543,0.000543,0.204132,0.000543,0.000543,0.014677
3,10364,0.001191,0.001191,0.459827,0.001191,0.001191,0.001191,0.001191,0.001191,0.001191,...,0.001191,0.001191,0.001191,0.220206,0.001191,0.001191,0.001191,0.001191,0.001191,0.001191
4,497,0.000412,0.019238,0.032279,0.105024,0.390893,0.000411,0.000412,0.015450,0.000412,...,0.000412,0.000412,0.000412,0.210518,0.000412,0.000412,0.000412,0.000412,0.000412,0.000411


In [61]:
customers.head()

,Unnamed: 0,account_id,district_id,frequency,date,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,0,576,55,0,1993-01-01,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894
1,1,704,55,0,1993-01-01,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894
2,2,192,55,0,1993-01-08,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894
3,3,10364,55,0,1993-01-17,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894
4,4,497,55,0,1993-04-15,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894


In [62]:
data = pd.merge(customers, types, on='account_id')[['account_id', 'A4', 'A11', 'A13', 'A16']+names]

In [63]:
data.shape

(4499, 30)

In [64]:
transaction.head()

,Unnamed: 0,account_id,trans_id,category,k-symbol,date,amount,mean_income,Initial Balance
0,0,1,5,0,-1,1995-03-24,122.633333,1358.788889,33.333333
1,1,1,199,29,-1,1995-04-13,420.000000,1358.788889,33.333333
2,2,1,3530438,40,2,1995-04-23,0.640000,1358.788889,33.333333
3,3,1,6,0,-1,1995-04-30,122.633333,1358.788889,33.333333
4,4,1,200,46,-1,1995-05-13,70.000000,1358.788889,33.333333


In [65]:
data = pd.merge(data, transaction, on='account_id', how='left')[['account_id', 'mean_income','A4', 'A11', 'A13', 'A16']+names]

In [66]:
data.shape

(1051771, 31)

In [67]:
data.drop_duplicates(inplace=True)

In [68]:
data.shape

(4499, 31)

In [69]:
data['A16'] = data['A16']/data['A4']

In [ ]:
data = data[data['amount']>0]

In [70]:
x = data[['mean_income','A4','A11','A13','A16']].as_matrix()
y = data[names].as_matrix()

In [71]:
print(x.shape, y.shape)

(4499, 5) (4499, 25)


In [72]:
import numpy, sys, json, hashlib, time

import keras
import keras.models, keras.layers.recurrent
from keras.layers.core import Dense
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers.normalization import BatchNormalization
from six.moves import range
from six import print_

input_dim=5
nSamples=x.shape[0]
output_dim=y.shape[1]
mid_level_width=100
#x=numpy.random.uniform(size=(nSamples,input_dim))
#y=numpy.zeros((nSamples,output_dim))
nn=keras.models.Sequential()
nn.add(Dense(mid_level_width,input_shape=(input_dim,),activation="relu"))
nn.add(Dense(mid_level_width,activation="relu"))
nn.add(Dense(output_dim,activation="softmax"))
op = keras.optimizers.Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
#nn.compile(loss='mse', optimizer=op,metrics = ["accuracy"])
nn.compile(loss='categorical_crossentropy', optimizer=op,metrics = ["accuracy"])
nn.summary()

nn.fit(x,y,nb_epoch=45,shuffle=0,batch_size=134)
print ("exiting at "+ time.strftime("%Y-%m-%d %H:%M:%S"))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               600       
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 25)                2525      
Total params: 13,225
Trainable params: 13,225
Non-trainable params: 0
_________________________________________________________________
Epoch 1/45


/home/ayman/anaconda3/envs/ati-dsg/lib/python3.6/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


4499/4499 [==============================] - 0s - loss: 15.9196 - acc: 0.0069       
Epoch 2/45
4499/4499 [==============================] - 0s - loss: 15.9196 - acc: 0.0069       
Epoch 3/45
4499/4499 [==============================] - 0s - loss: 15.9196 - acc: 0.0069       
Epoch 4/45
4499/4499 [==============================] - 0s - loss: 15.9196 - acc: 0.0069       
Epoch 5/45
4499/4499 [==============================] - 0s - loss: 15.9196 - acc: 0.0069       
Epoch 6/45
4499/4499 [==============================] - 0s - loss: 15.9196 - acc: 0.0069       
Epoch 7/45
4499/4499 [==============================] - 0s - loss: 15.9196 - acc: 0.0069       
Epoch 8/45
4499/4499 [==============================] - 0s - loss: 15.9196 - acc: 0.0069       
Epoch 9/45
4499/4499 [==============================] - 0s - loss: 15.9196 - acc: 0.0069       
Epoch 10/45
4499/4499 [==============================] - 0s - loss: 15.9196 - acc: 0.0069       
Epoch 11/45
4499/4499 [===========================

In [ ]:
nn.predict(x)